In [1]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh 

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 2.69 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 71.5 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-10-06 07:17:05--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c0:3470, 2406:da00:ff00::22cd:e0db, ...
Connecting to bitbucket.org (bi

In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable 
import pickle 

import MeCab
tagger = MeCab.Tagger()
#local에서 실행할 경우 참고사항 : 사전 디렉토리를 인자로 전달할 수 있음
#tagger = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ko-dic')

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Data setting

In [4]:
df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/aivle/data/college/college_FAQ.csv',encoding='utf-8', on_bad_lines='skip')


In [5]:
df.tail()

,id,category,question,answer
294,8,재수강,재수강한 과목의 성적이 더 안 좋을 때는 어떻게 되나요?,과목 재수강은 성적에 관계없이 이전에 수강한 과목의 성적이 삭제되고 새로이 수강한 ...
295,8,재수강,재수강 신청을 철회하려면 어떻게 하나요?,과목 재수강을 포기하고자 하는 경우에도 일반 수강신청 철회 기간 중에 철회 가능합니...
296,8,재수강,재수강하려는 과목이 개설되지 않았습니다. 어떻게 하나요?,삭제되는 과목에 대해서 대체 재수강 과목이 지정될 수 있습니다. 매 학기 수강신청 ...
297,8,재수강,대체 재수강을 운영하는 이유는 무엇인가요?,대체 재수강은 교과과정의 변경에 따라 삭제된 교과목을 학생들이 재수강할 수 있도록 ...
298,8,재수강,대체 재수강 신청 시 주의사항은 무엇인가요?,대체 재수강할 과목을 선택하여야만 재수강이 됩니다. 그렇지 않을 경우 새로운 과목을...


In [6]:
df.category.value_counts()

등록           80
휴복학          30
생활관          30
장학금          30
수강신청         30
계절제수업        30
증명서          29
복수전공/연계전공    20
재수강          20
Name: category, dtype: int64

In [7]:
subset_category=['등록','휴복학','수강신청','장학금','생활관']

df=df[df.category.isin(subset_category)]
# 학습데이터 원문 확인
print (df.head())
print()
print (df.tail())
print()


   id category                                           question  \
0   0       등록    대학원 수료생(당 학기 논문제출 예정자)의 경우 등록금 및 생활비 대출이 가능한가요?   
1   0       등록                  비학위과정의 대학원생은 등록금 및 생활비 대출이 가능한가요?   
2   0       등록  4년제 대학의 9학기, 2년제의 5학기 등과 같은 학기초과자 학생도 학자금 대출을 ...   
3   0       등록        학생 평점이 백분위 환산점수로 79.7점입니다. 든든학자금 대출이 가능한가요?   
4   0       등록                     재입학생이나 편입생은 재학생군인가요, 신입생 군인가요?   

                                              answer  
0   대학원 수료생의 경우 일부 금액이 등록금 개념으로 고지된다 하더라도 이 비용은 학...  
1         학점 인정을 떠나 비학위과정의 재학생은 대출자격요건에 충족되지 않습니다.\r  
2   원칙적으로 취업을 위한 의도적 졸업유예의 경우 학자금 대출이 불가합니다. 다만 졸...  
3   든든학자금 대출의 성적 기준은 80점 이상입니다. 성적은 반올림 처리가 되지 않으...  
4                         재입학생과 편입생 모두 신입생군에 속합니다.\r  

     id category                          question  \
195   4     수강신청      부제가 다른 특강 과목의 수강신청은 어떻게 하나요?   
196   4     수강신청  석박사과정 학생도 학사과정 과목을 수강 신청할 수 있나요?   
197   4     수강신청             학기 초과자인데 수강신청할 수 없나요?   
198   4     수강신청      조기취업을 하는데요 수강신청을

In [8]:
def mecabsplit(mecab_tagger,inputs, pos):
    r=[]
    inputs = mecab_tagger.parse(inputs)
    t = inputs.split('\n')[:-2]
    for i in t:
        field = i.split('\t')
        if field[1].split(',')[-1].strip() is not '*':
            r.extend( [ (x.split('/')[0],x.split('/')[1]) for x in field[1].split(',')[-1].split('+') ] )
        else:
            r.append( (field[0],field[1].split(',')[0]) )
    if pos:
        return r
    else:
        return [ x[0] for x in r ]
    return r

In [9]:
content_vocab = {'unk':0}
intent_vocab={}
intent_list=[]

data_intent=''
intent_idx=0
vocab_idx=1

for line in df.category.unique():
    intent=line
    if intent not in intent_vocab:
        intent_vocab[intent]=intent_idx
        intent_list.append(intent)
        intent_idx +=1

for line in list(df.question):
    line = mecabsplit(tagger,line,False)
    for it in line:
        if it not in content_vocab:
            content_vocab[it] = vocab_idx # it : 단어
            vocab_idx +=1

In [10]:
print("vocab size:", len(content_vocab))
print(content_vocab)  # 단어 사전
print(intent_vocab)   # intent 사전
print("intent size:", len(intent_list))
print(intent_list)    # intent_list : FAQ 카테고리 리스트

vocab size: 515
{'unk': 0, '대학': 1, '원': 2, '수료': 3, '생': 4, '(': 5, '당': 6, '학기': 7, '논문': 8, '제출': 9, '예정': 10, '자': 11, ')': 12, '의': 13, '경우': 14, '등록': 15, '금': 16, '및': 17, '생활': 18, '비': 19, '대출': 20, '이': 21, '가능': 22, '하': 23, 'ᆫ가요': 24, '?': 25, '학위': 26, '과정': 27, '은': 28, '4': 29, '년': 30, '제': 31, '9': 32, ',': 33, '2': 34, '제의': 35, '5': 36, '등': 37, '과': 38, '같': 39, '초과': 40, '학생': 41, '도': 42, '학자': 43, '을': 44, '받': 45, '수': 46, '있': 47, '나요': 48, '평점': 49, '백': 50, '분위': 51, '환산': 52, '점수': 53, '로': 54, '79': 55, '.': 56, '7': 57, '점': 58, 'ᄇ니다': 59, '들': 60, '든': 61, '재': 62, '입학': 63, '이나': 64, '편입': 65, '재학': 66, '군': 67, '신입': 68, '고지': 69, '서': 70, '출력': 71, '어디': 72, '에서': 73, 'ᆯ': 74, '왜': 75, '안': 76, '되': 77, '기간': 78, '이전': 79, '에': 80, '미리미리': 81, '를': 82, '고': 83, '싶': 84, '어요': 85, '복학': 86, '신청': 87, 'ᆫ데': 88, '았': 89, '는데': 90, '금액': 91, '이상': 92, '게': 93, '나오': 94, 'ㅏ요': 95, '다음': 96, '소속': 97, '변경': 98, '가': 99, '어떻게': 100, '는지': 101, '알': 102, '계열':

## Modeling

<연습1> 다음과 같은 CNN 모델을 정의하세요
- 단어 벡터 차원 : 100
- 출력 채널의 수 : 20
- 커널: 1단어, 2단어, 3단어로 된 커널 3개 사용

In [13]:
class CNN_Text(nn.Module):
    
    def __init__(self, embed_num, class_num, ):
        super(CNN_Text, self).__init__()

        #--- <연습2> 아래 빈 코드를 채우세요
        V = embed_num
        D = 100         # 단어임베딩 차원 
        C = class_num   # 분류할 카테고리 갯수
        Ci = 1          # 입력 채널의 수
        Co = 20         # 출력 채널의 수
        Ks = [1,2,3]    # 커널(필터) 사이즈 정의
        #--- end

        self.embed = nn.Embedding(V, D)  # 단어 벡터를 임의의 값으로 초기화

        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])

        self.dropout = nn.Dropout(0.2)  # dropout 정의
        
        #--- <연습3> Fully-connected layer를 정의하세요
        self.fc1 = nn.Linear(len(Ks)*Co, C)                     
        #--- end

    def forward(self, x):
        # x : 학습데이터 파일의 한 문장
        # 입력문장이 [대학, 원, 수료, 생, 논문, 제출] => 
        # x: [1, 2, 3, 4, 8, 9]의 입력이 생성됨 (content_vocab 출력결과 참고)
        
        # (N, W) Integer => (W) => view(1, -1) => (1, W)
        x = self.embed(x)         # (N, W, D) (1문장, 최대 입력 단어 개수, D차원)
        # 문장 x의 단어 벡터값 가져옴
        #print (x.size())

        #--- <연습4> unsqueeze 함수를 호출하세요
        x = x.unsqueeze(1) # (N, Ci, W, D) -> Ci를 추가하여 unsqueeze (Text는 채널 1개, 이미지는 채널 3개)
        #--- end
        
        #print (x.size())
        
        # 컨볼루션 부분 : 입력문 별로 호출되어 커널을 적용하면서 컨볼루션 수행
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)
        
        #print(len(x))
        #print(x[0].size())
        #print(x[1].size())
        #print(x[2].size())
        
        # max pooling 부분. max pooling을 수행한 결과 벡터의 차원은? (=출력채널의 갯수)
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)
                
        x = torch.cat(x, 1)

        x = self.dropout(x)  # (N, len(Ks)*Co)
        
        # 결과값(logit)은 실수값 2개를 출력하며 이 중 큰값을 가진 쪽이 예측값
        logit = self.fc1(x)  # (N, C) 

        #print(logit)
        return logit


In [14]:
cnn = CNN_Text(vocab_idx,intent_idx)
print(vocab_idx, intent_idx) # 파일에 나타난 단어수, 의도 갯수(클래스)
optimizer = torch.optim.Adam(cnn.parameters())
cnn.train()    # train mode로 준비

515 5


CNN_Text(
  (embed): Embedding(515, 100)
  (convs1): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(1, 100), stride=(1, 1))
    (1): Conv2d(1, 20, kernel_size=(2, 100), stride=(1, 1))
    (2): Conv2d(1, 20, kernel_size=(3, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=60, out_features=5, bias=True)
)

## Training

In [54]:
epoch = 20

# save_loss = 100
# es_count = 5
# count = 0

for e in range(epoch):
    totalloss = 0
    
    for line in df.values:
        target=Variable(torch.LongTensor([intent_vocab[line[1]]]))

        optimizer.zero_grad()

        cont = []
        # 입력 문장을 형태소 단위의 토큰으로 분리
        line = mecabsplit(tagger,line[2],False)
        
        for it in line:
            # 형태소 토큰을 content_vocab을 기준으로 id 값으로 변환하여 입력 생성
            cont.append(content_vocab[it])
        
        # list 변수를 torch에서 사용가능한 data_type으로 변환
        # view 함수는 입력 텐서를 reshape하는 함수
        # [입력 토큰 개수] => [1, 입력 토큰 개수]
        # 
        cont = Variable(torch.LongTensor(cont)).view(1,-1)
        pred = cnn(cont)
        
        #--- <연습5> 정답값과 예측값의 오차를 계산하는 손실함수 cross_entropy를 호출하세요
        loss = F.cross_entropy(pred, target)
        #--- end

        totalloss += float(loss.data)
        loss.backward()  # 필터, 임베딩벡터 값, FC 레이어의 matrix 등 모든 값을 loss를 줄이는 방향으로 계산

        
        #--- <연습6> 파라미터값을 변경하는 함수를 호출하세요
        # write code here
        optimizer.step()
        #--- end

    # if loss < save_loss:
    #   save_loss = loss
    #   save_cnn = cnn
    #   count = 0

    # else:
    #   count += 1

    # if count >= es_count:
    #   print("Early Stopping")
    #   break   
          
    
    print(e,'epoch')    
    print('loss : {:.3f}'.format(totalloss))
    #print (totalloss)


0 epoch
loss : 0.097
1 epoch
loss : 0.179
2 epoch
loss : 0.091
3 epoch
loss : 0.682
4 epoch
loss : 1.821
5 epoch
loss : 11.169
6 epoch
loss : 0.249
7 epoch
loss : 0.080
8 epoch
loss : 0.080
Early Stopping


## Response

In [55]:
response = df.answer

In [18]:
with open('/content/gdrive/My Drive/Colab Notebooks/aivle/data/college/college_FAQ_answer.txt', 'r', encoding='utf-8') as response_file:
    response = [row.split('\t')[1] for row in response_file.readlines()[1:]]

## Test

In [19]:
test=[]
# intent_list는 FAQ 카테고리 리스트:
# ['등록', '휴복학', '수강신청', '장학금', '생활관']
for i in intent_list:
    subset_df=df.query('category=="{}"'.format(i))
    test.append(subset_df.question.values[0])
    print(subset_df.question.values[0])

대학원 수료생(당 학기 논문제출 예정자)의 경우 등록금 및 생활비 대출이 가능한가요?
동록금을 납부한 뒤 휴학을 하면 나중에 복학할 때 등록금을 납부하지 않아도 되나요?
생활관과 서울역을 연결하는 교통편이 있나요?
교환학생도 장학금을 받을 수 있나요?
편입생(일반, 학사편입)의 수강신청은 어떻게 하나요?


In [20]:
test

['대학원 수료생(당 학기 논문제출 예정자)의 경우 등록금 및 생활비 대출이 가능한가요?',
 '동록금을 납부한 뒤 휴학을 하면 나중에 복학할 때 등록금을 납부하지 않아도 되나요?',
 '생활관과 서울역을 연결하는 교통편이 있나요?',
 '교환학생도 장학금을 받을 수 있나요?',
 '편입생(일반, 학사편입)의 수강신청은 어떻게 하나요?']

<테스트> 다양한 학사관련 질문을 입력하여 의도분석을 정확하게 수행하는지 테스트

In [49]:
# your test : input questions
#test=['2학기 수강신청 기간 언제인가요?']
#test=['장학금 받으려면 몇 학점 이상이어야 되나요?']
test = ["등록 이후에 장학금 얼마나 받을수있나요?"]

In [59]:
for idx in list(test):
    line = mecabsplit(tagger,idx, False)
   
    cont = []
    for it in line:
        if it in content_vocab:
            cont.append(content_vocab[it])  # cont에는 입력문에 나타난 단어들의 index 저장됨
        else:
            cont.append(content_vocab['unk'])
    
    cont = Variable(torch.LongTensor(cont)).view(1,-1)
    pred = cnn(cont)
    #pred : (1, C)
    v,i = torch.max(pred,1) # i: 가장 큰 값이 들어있는 위치 인덱스
    
    print('input : ', line)
    print('intent : ', intent_list[int(i)])
    #print(int(i))
    print('answer : ', response[int(i)],'\n')

input :  ['등록', '이후', '에', '장학', '금', '얼마나', '받', '을', '수', '있', '나요', '?']
intent :  장학금
 

input :  ['등록', '이후', '에', '장학', '금', '얼마나', '받', '을', '수', '있', '나요', '?']
intent :  장학금
 

